In [3]:
import pandas as pd
import mysql.connector as mc
import numpy as np

In [3]:
prod = pd.read_csv(r'./dta_at_prod_id.csv', index_col=0)
sql = '''insert into Products(brand_at_prod_id, department_at_prod_id, prod_id, group_at_prod_id, module_at_prod_id, amount_at_prod_id, units_at_prod_id) values (%s, %s, %s, %s, %s, %s, %s)'''

C:\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
prod.fillna('NULL', inplace=True)
prod.loc[prod.amount_at_prod_id == 'NULL', 'amount_at_prod_id'] = -1.0  # set float NULL as -1 in order to insert into database

In [5]:
prod_tuples = list(prod[['brand_at_prod_id', 'department_at_prod_id', 'prod_id', 'group_at_prod_id', 'module_at_prod_id', 'amount_at_prod_id', 'units_at_prod_id']].itertuples(index=False, name=None))

In [88]:
mydb = mc.connect(
    host="localhost",
    user="root",
    passwd="12345abcde",
    database="db_customer_panel"
)

In [38]:
# Before import data, set max_allowed_packet=2048M and innodb_buffer_pool_size=2048M in mysql system file my.ini
sql = '''INSERT INTO Products(brand_at_prod_id, department_at_prod_id, prod_id, group_at_prod_id, module_at_prod_id, amount_at_prod_id, units_at_prod_id) VALUES (%s, %s, %s, %s, %s, %s, %s)'''
mydbcursor = mydb.cursor()

# In case that mysql connector could not upload large dataframe, split the dataframe and insert seperately
mydbcursor.executemany(sql, prod_tuples[:1000000])
mydbcursor.executemany(sql, prod_tuples[1000000:2000000])
mydbcursor.executemany(sql, prod_tuples[2000000:3000000])
mydbcursor.executemany(sql, prod_tuples[3000000:])
mydbcursor.execute('UPDATE Products SET amount_at_prod_id=NULL WHERE amount_at_prod_id<0')  # change float -1 to NULL
mydb.commit()

In [42]:
del prod, prod_tuples
hh = pd.read_csv(r'./dta_at_hh.csv', index_col=0)
sql = '''INSERT INTO Households(hh_id, hh_race, is_latinx, hh_income, hh_size,hh_zip_code,hh_state,hh_residence_type) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'''
hh_tuples = list(hh[['hh_id', 'hh_race', 'hh_is_latinx', 'hh_income', 'hh_size', 'hh_zip_code','hh_state','hh_residence_type']].itertuples(index=False, name=None))
mydbcursor = mydb.cursor()
mydbcursor.executemany(sql, hh_tuples)
print(f'affected {mydbcursor.rowcount} rows')
mydb.commit()

affected 39577 rows


In [89]:
# del hh, hh_tuples
# trips = pd.read_csv(r'./dta_at_TC.csv', index_col=0)
# trips.fillna(-1, inplace=True)  # set float NULL as -1 in order to insert into database
sql = '''INSERT INTO Trips(hh_id, TC_date, TC_retailer_code, TC_retailer_code_store_code, TC_retailer_code_store_zip3, TC_total_spent, TC_id) VALUES (%s, %s, %s, %s, %s, %s, %s)'''
# trips_tuples = list(trips[['hh_id', 'TC_date', 'TC_retailer_code', 'TC_retailer_code_store_code', 'TC_retailer_code_store_zip3', 'TC_total_spent', 'TC_id']].itertuples(index=False, name=None))
mydbcursor = mydb.cursor()
mydbcursor.executemany(sql, trips_tuples[2:])
mydbcursor.execute('UPDATE Trips SET TC_retailer_code_store_zip3=NULL WHERE TC_retailer_code_store_zip3<0')  # change float -1 to NULL
print(f'affected {mydbcursor.rowcount} rows')
mydb.commit()

(2000000, '2003-12-29', 248, 4125185, 365.0, 3.97, 20576)

In [72]:
del trips, trips_tuples
tc_upc = pd.read_csv(r'./dta_at_TC_upc.csv', index_col=0)
sql = '''insert into Purchases(TC_id, quantity_at_TC_prod_id, total_price_paid_at_TC_prod_id, coupon_value_at_TC_prod_id, deal_flag_at_TC_prod_id, prod_id) values (%s, %s, %s, %s, %s, %s)'''
# tc_upc_tuples = list(tc_upc[['TC_id', 'quantity_at_TC_prod_id', 'total_price_paid_at_TC_prod_id', 'coupon_value_at_TC_prod_id', 'deal_flag_at_TC_prod_id', 'prod_id']].itertuples(index=False, name=None))
# mydbcursor = mydb.cursor()
# mydbcursor.executemany(sql, tc_upc_tuples)
# print(mydbcursor.rowcount)
# mydb.commit()

7596145

In [71]:
tc_upc = pd.read_csv(r'./dta_at_TC_upc.csv', index_col=0)
sql = '''insert into Purchases(TC_id, quantity_at_TC_prod_id, total_price_paid_at_TC_prod_id, coupon_value_at_TC_prod_id, deal_flag_at_TC_prod_id, prod_id) values (%s, %s, %s, %s, %s, %s)'''
tc_upc_tuples = list(tc_upc[['TC_id', 'quantity_at_TC_prod_id', 'total_price_paid_at_TC_prod_id', 'coupon_value_at_TC_prod_id', 'deal_flag_at_TC_prod_id', 'prod_id']].itertuples(index=False, name=None))
n = len(tc_upc_tuples)//1000000
for i in range(n):
    if i == n-1:
        mydbcursor.executemany(sql, tc_upc_tuples[i*1000000:])
        print(f'affect {i*1000000 + mydbcursor.rowcount} rows')
    else:
        mydbcursor.executemany(sql, tc_upc_tuples[i*1000000:(i+1)*1000000])
        print(f'affect {(i+1)*1000000} rows')
    mydb.commit()
mydb.close()

C:\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(38587942, 6)